# Technician routing problem
### prepared by Matt Delventhal

This notebook implements both a fully-customized, and a Gurobi-based, solution to the Gurobi example problem originally found [here](https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html). [This post](https://mattdelventhal.com/post/technician_route/) on my website presents more details about the problem specification, discusses the logic behind the custom algorithm, and compares the performance of these two solutions with the original solution presented by Gurobi.

In [1]:
## Dependencies

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import prod
from itertools import product as cp
from itertools import permutations
import time

In [2]:
## Calculate a single technician's partial contribution to cost function
def obj_func_technician_contribution(jobs,jtimes,penalties):
    funcvalue = 0
    for j in range(len(jobs)):        
        funcvalue += jobs[j]['priority']*max(jtimes[j] + jobs[j]['duration'] - jobs[j]['due_time'],0)
        funcvalue += jobs[j]['priority']*penalties[0]*max(jobs[j]['a']-jtimes[j],0)
        funcvalue += jobs[j]['priority']*penalties[0]*max(jtimes[j] - jobs[j]['b'],0)
    return funcvalue

## Find (heuristically-optimal) start times for a set of jobs, in a particular order, for a single technician
def tech_job_baselevel_optimizer(jobs,origin,ttimes,maxtime,penalties,costfunc,abs_min=0):
    jobtimes = []
    objvals = []
    next_inds = [0]*len(jobs)
    n_permutations = 3*(7**(len(jobs)-1))
    ttimelist = []
    cur_loc = str(origin)
    for job in jobs:
        ttimelist.append(float(ttimes.loc[(cur_loc,job['location']),'time']))
        cur_loc = str(job['location'])
    ttimelist.append(float(ttimes.loc[(cur_loc,origin),'time']))
        
    
    for it in range(n_permutations):
        ## initialize minimum objective value
        minobj = float("inf")
        
        ## update alignment options for current permutation
        cur_inds = [x for x in next_inds]
        
        ## initialize forward pass
        cur_fpass_loc = str(origin)
        cur_endmark = 0
        cuf_f_jtimes = []
        relevant = True
        
        ## loop simultaneously performs forward pass and iterates permutation
        for j in range(len(jobs)):
            #
            #
            iterate = False
            curmaxind = 6
            prevmaxind = 6
            if j == len(jobs) - 1:
                curmaxind = 2

            if j == 0:
                iterate = True
            elif (cur_inds[j-1] == prevmaxind) and (next_inds[j-1] == 0):
                iterate = True
            
            if iterate == True:
                if cur_inds[j] == curmaxind:
                    next_inds[j] = 0
                else:
                    next_inds[j] += 1
            
            fjob = jobs[j]
            next_fjob = jobs[min(len(jobs)-1,j+1)]
            cur_f_ttime = ttimelist[j]
            next_f_ttime = ttimelist[j+1]
            
            cur_endmark += cur_f_ttime
            
            if cur_inds[j] == 0:
                # start at latest point in window
                cur_endmark = max(cur_endmark,fjob['b'])
            elif cur_inds[j] == 1:
                # start to finish just in time
                if cur_endmark >= fjob['a']:
                    relevant = False
                    break
                else:
                    cur_endmark = max(cur_endmark,fjob['due_time']-fjob['duration'])
            elif cur_inds[j] == 2:
                # start at earliest point in window
                if cur_endmark >= fjob['b']:
                    relevant = False
                    break
                else:
                    cur_endmark = max(cur_endmark,fjob['a'])
            elif cur_inds[j] == 3:
                # 
                if next_fjob['b']-fjob['duration']-next_f_ttime >= cur_endmark + fjob['duration']:
                    relevant = False
                    break
                else:
                    cur_endmark = max(cur_endmark,next_fjob['b']-fjob['duration']-next_f_ttime)
            elif cur_inds[j] == 4:
                #
                if next_fjob['b']-fjob['duration']-next_f_ttime >= cur_endmark + fjob['duration']:
                    relevant = False
                    break
                else:
                    cur_endmark = max(cur_endmark,next_fjob['due_time']-next_fjob['duration']-fjob['duration']-next_f_ttime)
            elif cur_inds[j] == 5:
                #
                if next_fjob['b']-fjob['duration']-next_f_ttime >= cur_endmark + fjob['duration']:
                    relevant = False
                    break
                else:
                    cur_endmark = max(cur_endmark,next_fjob['a']-fjob['duration']-next_f_ttime)
            elif cur_inds[j] == 6:
                # finally, start as early as possible
                if cur_endmark > fjob['b']:
                    relevant = False
                    break
            cuf_f_jtimes.append(cur_endmark)
            cur_endmark += fjob['duration']
            cur_fpass_loc = str(fjob['location'])
            
            
        if relevant:
            curobj = costfunc(jobs,cuf_f_jtimes,penalties)
            #print(cur_inds,'relevant',curobj)
            if curobj < minobj:
                minobj = 1*curobj
                jobtimes = cuf_f_jtimes.copy()
        if minobj <= abs_min:
            break
        #else:
            #print('irrelevant')
    return curobj,jobtimes

## Iterate indices for which pre-calculated job assignment to try for each technician
def iterate_indices(inds_in,maxinds,iterate_from):
    inds_out = inds_in.copy()
    for j in range(len(inds_in)):
        iterate = False
        if j < len(inds_in) - iterate_from:
            inds_out[-1-j] = 0
        elif j == len(inds_in) - iterate_from:
            iterate = True
        elif inds_out[-j] == 0 and inds_in[-j] > 0:
            iterate = True
        if iterate:
            if inds_out[-1-j] == maxinds[-1-j] - 1:
                inds_out[-1-j] = 0
            else:
                inds_out[-1-j] += 1
    return inds_out


In [3]:
## Load data for the problem instance, except for set of jobs to be completed

technicians = pd.read_csv("technicians.csv",index_col=["tech_name"])
jobtypes = pd.read_csv("jobtypes.csv")
tech_quals = pd.read_csv("tech_quals.csv",index_col=["tech_name","job_type"])
ttimes = pd.read_csv("ttimes.csv",index_col=["origin","destin"])


In [4]:
## Parameters

daystart_h = 7 # start of planning horizon, in 24-hour time, within a single day
dayend_h = 17  # end of planning horizon, in 24-hour time, within a single day
M_penalties = [61,6100] # [penalty for missing job start window, penalty for leaving job undone]

In [5]:
## Load a particular set of jobs to be completed within a single day

joblist_df = pd.read_csv("job_list_01.csv")


## Merge in job type data for each job
joblist_df = joblist_df.merge(jobtypes,how="left",on="job_type")

## Count and list technicians qualified to do each job, and add to dataframe
qualled_techs = [list(tech_quals.loc[[(tech,joblist_df['job_type'][j]) for tech in technicians.index],
                                                 'qual']) \
                                  for j in range(len(joblist_df))]
number_qualified = [sum(qualtechs) for qualtechs in qualled_techs]
joblist_df["qualled_techs"] = qualled_techs
joblist_df["number_qualified"] = number_qualified


In [6]:
## Sort jobs by a rough measure of how much priority should be given to filling them. 
#     First, jobs with fewer qualified technicians are expected to be harder to fill.
#     Then, jobs with higher job priority
joblist_df = joblist_df.sort_values(by=['number_qualified','priority','due_time'],ascending=[True,False,True])
joblist_df = joblist_df.reset_index()


## Construct a list of jobs to be done, in order of their technician assignment priority

joblist = []
for j in range(len(joblist_df)):
    curjob = {}
    curjob['customer_id'] = joblist_df['customer_id'][j]
    curjob['location'] = joblist_df['location'][j]
    curjob['job_type'] = joblist_df['job_type'][j]
    curjob['due_time'] = (joblist_df['due_time'][j] - daystart_h)*60
    curjob['a'] = (joblist_df['early_start'][j] - daystart_h)*60
    curjob['b'] = (joblist_df['late_start'][j] - daystart_h)*60
    curjob['priority'] = joblist_df['priority'][j]
    curjob['duration'] = joblist_df['duration'][j]
    curjob['qualled_techs'] = joblist_df['qualled_techs'][j]
    curjob['number_qualified'] = joblist_df['number_qualified'][j]
    joblist.append(curjob)

In [7]:
## Sort technicians, putting those with ability to fill high-priority-to-fill jobs up front

technicianlist = np.array([technicians.index]).flatten()
list_of_techlists = [list(technicianlist[np.array(job['qualled_techs']) == 1]) for job in joblist]

prioritized_technicianlist = []
for tlist in list_of_techlists:
    for tech in tlist:
        if not tech in prioritized_technicianlist:
            prioritized_technicianlist.append(tech)

## Apply sorting to dataframe of technicians
technicians = technicians.loc[prioritized_technicianlist]

## Re-do list of qualified technicians for each job, using new ordering of technicians
for job in joblist:
    job["qualled_techs"] = list(tech_quals.loc[[(tech,job['job_type']) for tech in technicians.index],
                                                 'qual'])

## Construct list of technicians, using job-priority-based ordering

jobtype_list = [job['job_type'] for job in joblist]
techlist = []
for tech in technicians.index:
    curtech = {}
    curtech['tech_name'] = tech
    curtech['work_alloc'] = technicians['work_alloc'][tech]
    curtech['depot'] = technicians['depot'][tech]
    curtech['quals'] = [int(tech_quals.loc[(tech,jtype)]) for jtype in jobtype_list]
    techlist.append(curtech)

In [8]:
%%time

null_assignment = {}
null_assignment['assignments'] = []
null_assignment['jtimes'] = []
null_assignment['partial_value'] = 0

countpermute_potential = 0
countpermute_actual = 0

for tech in techlist:
    assigned_combos = []
    assigned_vals = []
    curorigin = str(tech['depot'])
    curquals = np.arange(len(joblist))[np.array(tech['quals']) == 1]
    tech['possible_assignments'] = []
    tech['possible_assignments'].append(null_assignment)
    
    permutelist = []
    for i in range(len(curquals)):
        permutelist += list(permutations(curquals,r=i+1))
            
    for permutation in permutelist:
        skipthisone = False
        if set(permutation) in assigned_combos:
            ind = assigned_combos.index(set(permutation))
            if assigned_vals[ind] == 0:
                skipthisone = True
        if not skipthisone:
            curjobs = [joblist[x] for x in permutation]
            timeest = sum([float(job['duration']) for job in curjobs]) \
                        + float(ttimes.loc[(tech['depot'],curjobs[0]['location']),'time']) \
                        + float(ttimes.loc[(curjobs[-1]['location'],tech['depot']),'time']) \
                        + sum([float(ttimes.loc[(curjobs[x]['location'],curjobs[min(len(curjobs)-1,x+1)]['location']),
                                                'time']) \
                                                                                for x in range(len(curjobs))])
            
            if timeest < tech['work_alloc']:
                curobj,curjtimes = tech_job_baselevel_optimizer(curjobs,curorigin,ttimes,tech['work_alloc'],\
                                                                M_penalties,obj_func_technician_contribution)
                if curobj < float("inf"):
                    curassignment = {}
                    curassignment['assignments'] = permutation
                    curassignment['jtimes'] = curjtimes
                    curassignment['partial_value'] = curobj
                    if set(permutation) in assigned_combos:
                        ind = assigned_combos.index(set(permutation))
                        if curobj < assigned_vals[ind]:
                            assigned_vals[ind] = 1*curobj
                            tech['possible_assignments'][ind+1] = curassignment.copy()
                    else:
                        assigned_combos.append(set(permutation))
                        assigned_vals.append(curobj)
                        tech['possible_assignments'].append(curassignment)
                        countpermute_actual += 1
                
                
    countpermute_potential += len(permutelist)
    
    #

print("Stored",countpermute_actual,"relevant out of",countpermute_potential,
      "theoretically-possible technician-specific permutations of job assignments.")

Stored 95 relevant out of 2394 theoretically-possible technician-specific permutations of job assignments.
CPU times: user 1.76 s, sys: 0 ns, total: 1.76 s
Wall time: 1.75 s


In [9]:
%%time

## Algorithm will stop looking if it finds a cost-function this low.
#    In this case, a negative cost is theoretically impossible.
abs_min = 0

## Initialize best-yet cost function
best_obj = float("inf")

## Initialize best-yet set of assignment indices
cur_best_inds = [0]*len(techlist)

## Initialize vector of assignment indices
allocation_inds = [0]*len(techlist)

## Number of relevant potential assignments for each technician, which will be used to tell the iterating 
#     algorithm when to "reset" the pointer or each technician to 0.
maxinds = [len(tech['possible_assignments']) for tech in techlist]

## Initialize some "administrative" variables, counting the number of naively-constructed allocations aborted 
#     because they either assign the same job to more than one technician,
#     or because they are "bounded out": because the accumulated partial cost exceeds the best-yet cost function
inconsistent_count = 0
boundhit_count = 0



st = time.time()
for it in range(prod(maxinds)):
    loud = False
    
    ## Run the iteration "loudly" every 500,000th time
    if it//500000 == it/500000:
        loud = True
    if loud:
        print('---')
        print('Iteration: ',it)
    cur_combined_obj = 0
    unalloc_jobpool = list(range(len(joblist)))
    aborted = False
    for k in range(len(techlist)):
        curtech = techlist[k]
        for job in curtech['possible_assignments'][allocation_inds[k]]['assignments']:
            if not job in unalloc_jobpool:
                if loud: 
                    print('inconsistent!')
                inconsistent_count += 1
                aborted = True
                break
            else:
                unalloc_jobpool.remove(job)
        if aborted:
            break
        cur_combined_obj += curtech['possible_assignments'][allocation_inds[k]]['partial_value']
        if cur_combined_obj >= best_obj:
            if loud: 
                print('hit bound!')
            boundhit_count += 1
            aborted = True
            break
    if not aborted:
        for jobn in unalloc_jobpool:
            cur_combined_obj += M_penalties[1]*joblist[jobn]['priority']
        if loud:
            et = time.time()
            print("Current best:",best_obj," | Most recent:",cur_combined_obj," | Elapsed:",et-st)
        if cur_combined_obj < best_obj:
            best_obj = cur_combined_obj*1
            cur_best_inds = [x for x in allocation_inds]
        if best_obj == abs_min:
            print("Absolute minimum achieved after ",it,"iterations")
            break
    allocation_inds = iterate_indices(allocation_inds,maxinds,k)
        
print(best_obj)    
for k in range(len(techlist)):
    print(techlist[k]['tech_name'] + ":",techlist[k]['possible_assignments'][cur_best_inds[k]]['assignments'])
print(inconsistent_count)
print(boundhit_count)


---
Iteration:  0
Current best: inf  | Most recent: 97600  | Elapsed: 0.00032448768615722656
Absolute minimum achieved after  719 iterations
0.0
Doris: []
Gina: (0,)
Flor: (1, 2)
Albert: (4, 5)
Carlos: []
Ed: (6, 3)
Bob: []
268
8
CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 10.7 ms


In [10]:
## Import Gurobi. 
#     Note that the free license might not be enough for this problem--it may be over the variable limit.

import gurobipy as gp
from gurobipy import GRB

In [11]:
## Define data structure for Gurobi solution implementation

Kset = [tech['tech_name'] for tech in techlist]
Jset = [job['customer_id'] for job in joblist]
Loc = {job['customer_id']:job['location'] for job in joblist}
a = {job['customer_id']:job['a'] for job in joblist}
b = {job['customer_id']:job['b'] for job in joblist}
c = {job['customer_id']:job['due_time'] for job in joblist}
p = {job['customer_id']:job['duration'] for job in joblist}
pi = {job['customer_id']:job['priority'] for job in joblist}
Origin = {tech['tech_name']:tech['depot'] for tech in techlist}
Tau = {ind:float(ttimes.loc[ind,'time']) for ind in ttimes.index}
qprep = [{(job['customer_id'],techlist[i]['tech_name']):job['qualled_techs'][i] for i in range(len(job['qualled_techs']))} for job in joblist]
q = {}
for qitem in qprep:
    q |= qitem
W = {tech['tech_name']:tech['work_alloc'] for tech in techlist}
T = (dayend_h - daystart_h)*60

R = len(Jset)



In [12]:
## instantiate model
m = gp.Model()

## declare decision variables
x = m.addVars(Jset,Kset,range(R),vtype=GRB.BINARY,name="x")
t = m.addVars(Jset,ub=T,name="t")

## declare and constrain auxiliary variables
XX_same_r = m.addVars(Jset,Jset,Kset,range(R),vtype=GRB.BINARY,name="XX_same_r")
m.addConstrs(((XX_same_r[i,j,k,r] == gp.min_(x[i,k,r],x[j,k,r])) for (i,j,k,r) in cp(Jset,Jset,Kset,range(R)) ),
             name="XX_same_r_indic")

XX_offset_r = m.addVars(Jset,Jset,Kset,range(1,R),vtype=GRB.BINARY,name="XX_offset_r")
m.addConstrs(((XX_offset_r[i,j,k,r] == gp.min_(x[i,k,r-1],x[j,k,r])) for (i,j,k,r) in cp(Jset,Jset,Kset,range(1,R)) ) 
             ,name="XX_offset_r_indic")

lastStop = m.addVars(Jset,Kset,range(R),vtype=GRB.BINARY,name="lastStop")
m.addConstrs((lastStop[i,k,r] == gp.quicksum(XX_same_r[i,j,k,r] for j in Jset)\
                                 *(x[i,k,r] - gp.quicksum(XX_offset_r[i,j,k,r+1] for j in Jset)) \
              for (i,k,r) in cp(Jset,Kset,range(R-1)))  )
m.addConstrs((lastStop[i,k,R-1] == x[i,k,R-1] for i,k in cp(Jset,Kset)),"lastStopindic" )

lateness = m.addVars(Jset,name="lateness")
m.addConstrs((lateness[j] >= t[j] + p[j] - c[j] for j in Jset),
             name="lateness_def"
            )

tooearlystart = m.addVars(Jset,name="tooearlystart")
m.addConstrs((tooearlystart[j] >= a[j] - t[j] for j in Jset),
             name="tooearlystart_indic"
            )

toolatestart = m.addVars(Jset,name="toolatestart")
m.addConstrs((toolatestart[j] >= t[j] - b[j] for j in Jset),
             name="toolatestart_indic"
            )


## declare constraints
m.addConstrs((gp.quicksum(x[j, k, r] for (k,r) in cp(Kset,range(R))) <= 1 for j in Jset),
             name="assign_atmost_one_tech")

m.addConstrs(( gp.quicksum(x[j,k,r] for r in range(R))*(1-q[j,k])  == 0 for (j,k) in cp(Jset,Kset)),
             name="assign_only_if_qualified")

m.addConstrs((gp.quicksum(x[j,k,0] for j in Jset) <= 1 for k in Kset),name="each_tech_atmost_one_first_job")


m.addConstrs((gp.quicksum(x[j,k,r] for j in Jset) <= gp.quicksum(x[j,k,r-1] for j in Jset) \
              for (k,r) in cp(Kset,range(1,R))),
             name="no_gaps_in_assignment_of_job_order_for_each_tech")

m.addConstrs((t[j] >= gp.quicksum(Tau[Origin[k],Loc[j]]*x[j,k,0] for k in Kset) \
                      + gp.quicksum(XX_offset_r[i,j,k,r]*(t[i] + p[i] + Tau[Loc[i],Loc[j]]) \
                                    for (i,k,r) in cp(Jset,Kset,range(1,R))) \
              for j in Jset),name="lowerbound_jobstart")

m.addConstrs((gp.quicksum(x[j,k,r]*p[j] for (j,r) in cp(Jset,range(R))) \
              + gp.quicksum(x[j,k,0]*Tau[Origin[k],Loc[j]] for j in Jset) \
              + gp.quicksum(XX_offset_r[i,j,k,r]*Tau[Loc[i],Loc[j]] for (i,j,r) in cp(Jset,Jset,range(1,R))) \
              + gp.quicksum(lastStop[i,k,r]*Tau[Loc[i],Origin[k]] for (i,r) in cp(Jset,range(R)) ) \
              <= W[k] for k in Kset)  ,"worktime_constr")

m.setObjective(gp.quicksum(pi[j]*(1-gp.quicksum(x[j,k,r] for (k,r) in cp(Kset,range(R))))*M_penalties[1] \
                           + gp.quicksum(x[j,k,r] for (k,r) in cp(Kset,range(R)))*(lateness[j]*pi[j] \
                           + tooearlystart[j]*pi[j]*M_penalties[0] \
                           + toolatestart[j]*pi[j]*M_penalties[0])
                          for j in Jset), GRB.MINIMIZE)


Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-16


In [13]:
# Solve it!
m.optimize()

print(f"Optimal objective value: {m.objVal}")

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 24 physical cores, 48 logical processors, using up to 24 threads
Optimize a model with 182 rows, 5173 columns and 3731 nonzeros
Model fingerprint: 0xaf002fe6
Model has 1029 quadratic objective terms
Model has 301 quadratic constraints
Model has 4459 general constraints
Variable types: 28 continuous, 5145 integer (5145 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+02]
  Objective range  [6e+03, 2e+04]
  QObjective range [2e+00, 5e+02]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 5e+02]
Presolve added 1759 rows and 0 columns
Presolve removed 0 rows and 4259 columns
Presolve time: 0.17s
Presolved: 8104 rows, 4108 columns, 22814 nonzeros
Presolved model has 762 SOS constraint(s)
Variable types: 1196 continuous, 2912 integer (2912 binary)
Found heuristic solution: objective 97600.000000

Root relaxation: objective 0.000000